**June 4, 2025 • 10:23 AM**

In [ ]:
#!/usr/bin/env python3
# Panutad Sirikul (French)
# 650510714
# user_activity
# 229496 Sec DS

วันนี้อากาศดี ⛅ ฉันนัดวันเวลานิเทศครั้งที่ 1 กับอาจารย์ที่ปรึกษาสหกิจหลักและอาจารย์ที่ปรึกษาสหกิจร่วมและพี่เลี้ยง เรียบร้อย ส่วนเรื่องอื่น ๆ คือ เพื่อนยอร์ชพึ่งจัด Elden Ring มาแล้ว เราคงต้องรีบไปหาดาบ Nagakiba มาให้สายซามูไรอย่างเราโดยเร็วแล้ว

# <b>7th Week's Assignment</b>
งานวันนี้ Wednesday, June 4, 2025 (หลังจาก HELLDIVERS™ 2 - EP.287-EP.289 ล่าแมลง 3 Operations เมื่อคืน)

หลัก ๆ น่าจะใช้ Python, Streamlit, VS Code ในการทำ
## <b>1. Plot Histogram</b>
ทำการ Plot Histogram หา Date Difference ของลูกค้า (ผู้ใช้บริการ) โดยการ date_diff = first_use_date - regist_date

Note: ไม่ต้องคำนวณเป็นรายชั่วโมง ให้ดูเป็นวันไปเลย เช่น สมัครวันที่ 2025-06-04 และใช้บริการครั้งแรกในวันเดียวกัน จะได้ date_diff เป็น 0 
แต่ถ้าสมมติว่า สมัครตอน 5 ทุ่ม แล้วจองครั้งแรกตอนตี 1 ของวันถัดไป จะได้ date_diff เป็น 1 ไปเลย


### <b>Importing Dataset</b>

<b>Let's take a look at our dataset — user_first_booking_raw.csv</b>


In [71]:
# importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [72]:
# ignore warning & show all columns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [73]:
# Importing Dataset
df = pd.read_csv("user_first_booking_raw.csv")
df.head()

,user_id,registed_date,first_booking_date
0,1,2018-06-15 14:41:51,2018-07-31 18:09:36
1,2,2016-03-22 23:32:57,NaN
2,3,2018-06-15 16:54:04,2016-03-24 21:14:38
3,9,2016-03-23 14:25:07,NaN
4,10,2016-03-29 14:02:51,2018-07-31 18:12:57


## <b>Preprocessing and EDA:<b>

<div class="alert alert-block alert-info">
<b>Note:</b> Dataset is already cleaned, but I should check it again for clarity. After that is done, the data should be ready to use for finding date difference (date_diff = first_use_date - regist_date)</div>

In [74]:
df.columns

Index(['user_id', 'registed_date', 'first_booking_date'], dtype='object')

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401709 entries, 0 to 401708
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   user_id             401709 non-null  int64 
 1   registed_date       401709 non-null  object
 2   first_booking_date  162715 non-null  object
dtypes: int64(1), object(2)
memory usage: 9.2+ MB


In [76]:
df.tail()

,user_id,registed_date,first_booking_date
401704,403911,2025-06-04 08:08:07,NaN
401705,403912,2025-06-04 08:11:06,NaN
401706,403913,2025-06-04 08:15:49,NaN
401707,403914,2025-06-04 08:17:47,NaN
401708,403915,2025-06-04 08:29:13,NaN


<b>Rename column registed_date to register_date</b>

In [77]:
df.rename(columns={"registed_date": "register_date"}, inplace=True)
df.head()

,user_id,register_date,first_booking_date
0,1,2018-06-15 14:41:51,2018-07-31 18:09:36
1,2,2016-03-22 23:32:57,NaN
2,3,2018-06-15 16:54:04,2016-03-24 21:14:38
3,9,2016-03-23 14:25:07,NaN
4,10,2016-03-29 14:02:51,2018-07-31 18:12:57


<b>Register_date and first_booking_date to have only date part</b>

In [78]:
# Keep only the date part for register_date and first_booking_date (if not NaN)
df["register_date"] = df["register_date"].astype(str).str.split().str[0]

# Only filter first_booking_date if it's not NaN
df.loc[df["first_booking_date"].notna(), "first_booking_date"] = (
    df.loc[df["first_booking_date"].notna(), "first_booking_date"].astype(str).str.split().str[0]
)

df.head()

,user_id,register_date,first_booking_date
0,1,2018-06-15,2018-07-31
1,2,2016-03-22,NaN
2,3,2018-06-15,2016-03-24
3,9,2016-03-23,NaN
4,10,2016-03-29,2018-07-31


<b>Change datatype to datetime for relevant columns</b>

In [79]:
df["register_date"].isnull().sum()

np.int64(0)

In [80]:
df["first_booking_date"].isnull().sum()

np.int64(238994)

In [81]:
df["register_date"].unique()

array(['2018-06-15', '2016-03-22', '2016-03-23', ..., '2025-06-02',
       '2025-06-03', '2025-06-04'], shape=(3245,), dtype=object)

In [82]:
df["register_date"] = df["register_date"].str.strip()
df["register_date"] = pd.to_datetime(df["register_date"], errors='coerce')

### <b>Data Filtration</b>

The process of narrowing down relevant information from a large dataset using specific conditions or criteria. It simplifies data analysis by allowing you to focus on data that meets certain conditions while removing unnecessary information.

<b>1st Condition:</b> Remove rows where first_booking_date is NaN

Basically, remove a data row where user have registered but never use any of our services.

In [83]:
df = df[df["first_booking_date"].notna()]
df = df[df["register_date"].notna()]
df.head()

,user_id,register_date,first_booking_date
0,1,2018-06-15,2018-07-31
2,3,2018-06-15,2016-03-24
4,10,2016-03-29,2018-07-31
5,11,2016-03-29,2016-03-30
7,13,2018-06-02,2016-04-02


In [84]:
c1 = df["user_id"].count()
c2 = df["register_date"].count()
c3 = df["first_booking_date"].count()

print(f"user_id: {c1} rows")
print(f"register_date: {c2} rows")
print(f"first_booking_date: {c3} rows")

user_id: 162694 rows
register_date: 162694 rows
first_booking_date: 162694 rows


<b>2nd Condition:</b> "register_date" should always be less than or equal to "first_booking_date"

In [85]:
# Keep only rows where register_date <= first_booking_date
df = df[df["register_date"] <= df["first_booking_date"]]
df.head()

,user_id,register_date,first_booking_date
0,1,2018-06-15,2018-07-31
4,10,2016-03-29,2018-07-31
5,11,2016-03-29,2016-03-30
13,20,2016-04-03,2016-04-03
20,27,2016-04-10,2016-04-10


In [86]:
c1 = df["user_id"].count()
c2 = df["register_date"].count()
c3 = df["first_booking_date"].count()

print(f"user_id: {c1} rows")
print(f"register_date: {c2} rows")
print(f"first_booking_date: {c3} rows")

user_id: 159785 rows
register_date: 159785 rows
first_booking_date: 159785 rows


In [87]:
# reset the index and remove the old index from df
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159785 entries, 0 to 159784
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   user_id             159785 non-null  int64         
 1   register_date       159785 non-null  datetime64[ns]
 2   first_booking_date  159785 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 3.7+ MB


<b>3rd Condition:</b> user_id must be unique

In [90]:
# Check for duplicate user_id
print(f"user_id duplicated: {df.user_id.duplicated().sum()} row")

user_id duplicated: 0 row


## <b>Date Difference:</b>

<b>Create a new column "date_diff" as the number of days between first_booking_date and register_date</b>

In [92]:
df["first_booking_date"].dtypes

dtype('O')

In [93]:
# first_booking_date is still of type object
# need to explicitly convert it to datetime
df["first_booking_date"] = pd.to_datetime(df["first_booking_date"], errors="coerce")
df["first_booking_date"].dtypes

dtype('<M8[ns]')

In [95]:
df["date_diff"] = (df["first_booking_date"] - df["register_date"]).dt.days
df.head()

,user_id,register_date,first_booking_date,date_diff
0,1,2018-06-15,2018-07-31,46
1,10,2016-03-29,2018-07-31,854
2,11,2016-03-29,2016-03-30,1
3,20,2016-04-03,2016-04-03,0
4,27,2016-04-10,2016-04-10,0


<b>Save the cleaned DataFrame to a new CSV file</b>

In [96]:
df.to_csv("user_first_booking_cleaned.csv", index=False)

## <b>Data Visualization</b>

<b>Display an interactive histogram showing the frequency of each date_diff value in this DataFrame.</b>

In [97]:
#import plotly.express as px

# Plot histogram of date_diff using plotly
fig = px.histogram(df, x="date_diff", nbins=30, title="Frequency of Date Difference (date_diff)")
fig.update_layout(xaxis_title="Date Difference (days)", yaxis_title="Frequency")
fig.show()

In [102]:
#import plotly.express as px

# Filter to only show date_diff from 0 to 30 days
filtered_df = df[df['date_diff'].between(0, 30)]

# Calculate the count (frequency) of each date_diff value (0-30)
date_diff_count = filtered_df['date_diff'].value_counts().sort_index()
count_df = date_diff_count.reset_index()
count_df.columns = ['date_diff', 'count']

# Plot count of date_diff using plotly
fig = px.bar(count_df, x="date_diff", y="count", title="Total Users of Date Difference (0-30 days)")
fig.update_layout(xaxis_title="Date Difference (days)", yaxis_title="Total Users")
fig.show()

<b>Summary of Date Difference Distribution</b>

From the graph, we can see that a significant majority of users (around 88,000) registered and made their first booking on the same day (date_diff = 0). The number of users sharply decreases as the number of days between registration and first booking increases. This indicates that most users tend to use the service immediately after registering, while only a smaller portion waits several days before making their first booking.